In [11]:
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

In [23]:
training_data = Path().cwd() / "mri" / "Training"
testing_data = Path().cwd() / "mri" / "Testing"

In [ ]:
if training_data.exists() and testing_data.exists():
    print("ścieżki istnieją i można śmigać dalej")

    categories_paths = training_data.glob("*")
    categories = [category.name for category in categories_paths]
    print(categories)

    train_imgs_paths = training_data.glob("*/*.jpg")
    test_imgs_paths = testing_data.glob("*/*.jpg")

    training_imgs = [img.name for img in train_imgs_paths]
    testing_imgs = [img.name for img in test_imgs_paths]

    print(f"ilosc treningowych i walidacyjnych: {len(training_imgs)}; ilosc testowych: {len(testing_imgs)}")

else: 
    print('coś nie hula')

ścieżki istnieją i można śmigać dalej
['glioma', 'meningioma', 'notumor', 'pituitary']
ilosc treningowych: 5712; ilosc testowych: 1311


In [110]:
img_resizer = transforms.Resize([250, 250])
img_tensor = transforms.ToTensor()

img_transformer = transforms.Compose([img_resizer, img_tensor])
print(img_transformer)

Compose(
    Resize(size=[250, 250], interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
)


In [ ]:
full_training_dataset_transformed = datasets.ImageFolder(root=training_data, transform=img_transformer)
full_test_dataset_transformed = datasets.ImageFolder(root=testing_data, transform=img_transformer)
img, category = full_training_dataset_transformed[0]
print('--treningowe--')
print(f"rozmiar: {img.shape}; typ {type(img)}; kategoria: {category}")
img, category = full_test_dataset_transformed[0]
print('--testowe--')
print(f"rozmiar: {img.shape}; typ {type(img)}; kategoria: {category}")


--treningowe--
rozmiar: torch.Size([3, 250, 250]); typ <class 'torch.Tensor'>; kategoria: 0
--testowe--
rozmiar: torch.Size([3, 250, 250]); typ <class 'torch.Tensor'>; kategoria: 0


In [ ]:
seed = torch.Generator().manual_seed(42)
train_data, val_data = random_split(full_training_dataset_transformed, [0.75, 0.25], generator=seed)

In [112]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, pin_memory=True)  
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, pin_memory=True)
test_loader = DataLoader(full_test_dataset_transformed, batch_size=32, shuffle=False, pin_memory=True)

In [ ]:
# PIERWSZA PRÓBA

class AntiTumor(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten(),

            nn.Linear(32 * 62 * 62, 128),
            nn.ReLU(),

            nn.Linear(128, 4)
            )
        
    def forward(self, x):
        return self.layers(x)


In [105]:
model = AntiTumor()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Models utilizes {device}")

Models utilizes cuda


In [106]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [108]:
epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    model.train()

    for img, category in train_loader:
        img, category = img.to(device), category.to(device)
        optimizer.zero_grad()
        outputs = model(img)
        loss = loss_func(outputs, category)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoka [{epoch+1}/{epochs}] | Średni Błąd (Loss): {avg_loss:.4f}")

print("------KONIEC------")

Epoka [1/10] | Średni Błąd (Loss): 0.6974
Epoka [2/10] | Średni Błąd (Loss): 0.3040
Epoka [3/10] | Średni Błąd (Loss): 0.1634
Epoka [4/10] | Średni Błąd (Loss): 0.0934
Epoka [5/10] | Średni Błąd (Loss): 0.0385
Epoka [6/10] | Średni Błąd (Loss): 0.0135
Epoka [7/10] | Średni Błąd (Loss): 0.0032
Epoka [8/10] | Średni Błąd (Loss): 0.0023
Epoka [9/10] | Średni Błąd (Loss): 0.0006
Epoka [10/10] | Średni Błąd (Loss): 0.0004
------KONIEC------


In [109]:
correct = 0
total = 0

model.eval() # Przełączamy model w tryb OCENY (wyłącza Dropout itp.)

with torch.no_grad(): # Mówimy PyTorchowi: "Nie licz gradientów, oszczędzaj pamięć"
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Model przewiduje
        outputs = model(images)
        
        # Wybieramy klasę z największą liczbą punktów (logitów)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Skuteczność modelu na danych testowych: {accuracy:.2f}%')

Skuteczność modelu na danych testowych: 93.56%


In [113]:
model.eval() # Przełączamy model w tryb oceniania
poprawne = 0
wszystkie = 0

with torch.no_grad(): # Wyłączamy liczenie gradientów (oszczędzamy pamięć)
    for obrazy, etykiety in test_loader:
        obrazy, etykiety = obrazy.to(device), etykiety.to(device)
        
        wyniki = model(obrazy)
        _, przewidziane = torch.max(wyniki.data, 1)
        
        wszystkie += etykiety.size(0)
        poprawne += (przewidziane == etykiety).sum().item()

skutecznosc = 100 * poprawne / wszystkie
print(f'Ostateczna skuteczność na folderze testowym: {skutecznosc:.2f}%')

Ostateczna skuteczność na folderze testowym: 93.67%
